In [59]:
import numpy as np
import math

def trapezoidal_rule(f, a, b, n):
    """ 
    Transforms the integral K(x, t) * y(t) dt from a to b through the trapezoidal rule
    
    Parameters:
        K -> kennel K(x, t) given by us
        a -> float, inferior limit of the integral
        b -> float, superior limit of the integral
        n -> int, nr of intervals for the rule
    
    it returns an aproximated number of the integral
    """
    h = (b - a) / n
    integral = 0.5 * h * (f(a) + f(b))
    for i in range(1, n):
        integral += h * f(a + i * h)
    return integral

def midpoint_rule(f, a, b, n):
    """
    Transforms the integral K(x, t) * y(t) dt from a to b through the midpoint rule
    """
    h = (b - a) / n
    integral = 0
    for i in range(n):
        x_mid = a + (i + 0.5) * h
        integral += h * f(x_mid)
    return integral

def simpsons_rule(f, a, b, n):
    """
    Transforms the integral K(x, t) * y(t) dt from a to b through the simpsons rule
    """
    h = (b - a) / n
    integral = f(a) + f(b)
    for i in range(1, n, 2):
        integral += 4 * f(a + i * h)
    for i in range(2, n-1, 2):
        integral += 2 * f(a + i * h)
    return h * integral / 3


def nystrom_method(a, b, alpha, beta, f, K, n, quadrature_rule):
    """
    Solves an integro-diferencial in the form of:
    y''(x) + alpha(x) * y'(x) + beta(x) * y(x) + ∫[a,b] K(x, t) * y(t) dt = f(x), for x ∈ [a, b]
    with boundary conditions y(a) = y(b) = 0.
    
    Returns:
    x: the gridpoints of the interval [a, b]
    y: the numerical solution in the grid points xi
    """
    x = np.linspace(a, b, n)  # Divides [a, b] in n equaly separated points
    h = (b - a) / n  # size of the step for the finite differences equation
    
    # transforms the kennel K(x,t) in acordance with the quadrature rule
    Kernel = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            Kernel[i, j] = quadrature_rule(lambda t: K(x[i], t), a, b, n)
    
    #centred finite differences
    d_f_2=1/(h**2)
    d_f_1=1/(2*h)

    #progressive finite differences
    #d_f_2= 1/(h**2)
    #d_f_1=1 / h

    #regressive finite differences
    #d_f_2=1 / h**2
    #d_f_1=1 / h


    # Inicialization for the Nystrom matrix with zeros
    Nystrom_matrix = np.zeros((n, n))
    
    # Filling the Nystrom matrix
    for i in range(n):
        Nystrom_matrix[i, i] += d_f_2  # term y''(x) through finite differences
        if i > 0:
            Nystrom_matrix[i, i-1] += alpha(x[i]) * d_f_1  # term alpha(x) * y'(x) through finite differences
        Nystrom_matrix[i, i] += beta(x[i])  # term beta(x) * y(x) 
        for j in range(n):
            Nystrom_matrix[i, j] += Kernel[i, j]  # integral term after discretization
    
    # Adding the initial conditions y(a) = y(b) = 0
    Nystrom_matrix[0, :] = 0
    Nystrom_matrix[-1, :] = 0
    Nystrom_matrix[0, 0] = 1
    Nystrom_matrix[-1, -1] = 1

    
    # Creating the sol matrix in the right side
    sol = np.zeros(n)
    for i in range(1, n-1):
        sol[i] = f(x[i])
    sol[0] = 0
    sol[-1] = 0
    
    # solving the resulting linear system 
    y = np.linalg.solve(Nystrom_matrix, sol)
    
    return x, y

def convergence_level(a, b, alpha, beta, f, K, n, quadrature_rule):
    """
    Calculates the convergency of the Nystrom method through a reduction facter r=2
    parameters:
    n -> int, value of the first grid
    
    Returns:
    level -> float, value of the convergency level
    """
    
    
    u, v = nystrom_method(a, b, alpha, beta, f, K, n, quadrature_rule)
    w, z = nystrom_method(a, b, alpha, beta, f, K, n//2, quadrature_rule)
    m, p = nystrom_method (a, b, alpha, beta, f, K, 1, quadrature_rule)

    num1 = np.linalg.norm(v)
    num2 = np.linalg.norm(z)
    num3 = np.linalg.norm(p)    
    
    #calculamos o erro dos dois passos
    erro1=np.abs(num1-num3)
    #calculamos uma estimativa do erro para h2 (vamos assumir 2 ordem de conv)
    erro2=np.abs(num2-num3)

    
 
    return np.abs(np.log(erro1/erro2)/np.log(2))
   



"""# EXAMPLE 1:
def K(x, t):
    return np.sin(2 * x + t)  # Example kernel function

def alpha(x):
    return x**2  # Example function alpha(x)

def beta(x):
    return x+1  # Example function beta(x)

def f(x):
    return np.sin(x) # Example function f(x)

a = 0
b = math.pi
n = 100

# Define the quadrature rule (e.g., trapezoidal rule)
quadrature_rule = midpoint_rule"""



"""# EXAMPLE 2
a = -1
b = 2
n = 200

def f(x):
    #example function f(x)
    return x**3

def alpha(x):
    #example function alpha(x)
    return 2*x

def beta(x):
    #example function beta(x)
    return 2*x

def K(x, t):
    return x * t  # Example kernel function

quadrature_rule = simpsons_rule  # Choose the quadrature rule to use
"""

#Exemplo 3 
def K(x, t):
    if x != 0:
        return 1/(x-t)**3
    else: 
        return 0


a = -1
b = 2
n = 100

def f(x):
    #example function f(x)
    return x**3

def alpha(x):
    #example function alpha(x)
    return 2*x

def beta(x):
    #example function beta(x)
    return 2*x


quadrature_rule = midpoint_rule  # Choose the quadrature rule to use


x,y=nystrom_method(a, b, alpha, beta, f, K, n, quadrature_rule)
level = convergence_level(a, b, alpha, beta, f, K, n, quadrature_rule) 

print("Grid points x:\n", x)  # noqa: E999

print("\nNumerical solution y:\n", y)

print("\nNivel de convergencia:\n", level)







Grid points x:
 [-1.         -0.96969697 -0.93939394 -0.90909091 -0.87878788 -0.84848485
 -0.81818182 -0.78787879 -0.75757576 -0.72727273 -0.6969697  -0.66666667
 -0.63636364 -0.60606061 -0.57575758 -0.54545455 -0.51515152 -0.48484848
 -0.45454545 -0.42424242 -0.39393939 -0.36363636 -0.33333333 -0.3030303
 -0.27272727 -0.24242424 -0.21212121 -0.18181818 -0.15151515 -0.12121212
 -0.09090909 -0.06060606 -0.03030303  0.          0.03030303  0.06060606
  0.09090909  0.12121212  0.15151515  0.18181818  0.21212121  0.24242424
  0.27272727  0.3030303   0.33333333  0.36363636  0.39393939  0.42424242
  0.45454545  0.48484848  0.51515152  0.54545455  0.57575758  0.60606061
  0.63636364  0.66666667  0.6969697   0.72727273  0.75757576  0.78787879
  0.81818182  0.84848485  0.87878788  0.90909091  0.93939394  0.96969697
  1.          1.03030303  1.06060606  1.09090909  1.12121212  1.15151515
  1.18181818  1.21212121  1.24242424  1.27272727  1.3030303   1.33333333
  1.36363636  1.39393939  1.42424242

In [47]:
import pandas as pd

df = pd.DataFrame({'x': x, 'y': y})
print(df)
df.to_csv('nystrom.csv', index=False) # TO DO: Change the filename to your liking

           x             y
0  -1.000000  3.155371e-16
1  -0.969697 -6.866123e-04
2  -0.939394 -6.355086e-04
3  -0.909091 -5.676734e-04
4  -0.878788 -5.040119e-04
..       ...           ...
95  1.878788  5.646186e-03
96  1.909091  5.918423e-03
97  1.939394  6.199104e-03
98  1.969697  6.488346e-03
99  2.000000  0.000000e+00

[100 rows x 2 columns]
